# Test reading Logs

In [54]:
import pandas as pd
import re

In [10]:
df_log = pd.read_csv(
    './data/1_AC/o00406-0100700090001/logd.dat', 
    sep='\t', 
    header=None, 
    encoding='latin-1',
    names=['date', 'info', 'id1', 'info2', 'operation', 'id2']
)


## Cleaning data

Convertendo datas para o formato datetime

In [19]:
# convert date to datetime
df_log['date'] = pd.to_datetime(df_log['date'], format='%d/%m/%Y %H:%M:%S')

df_log.loc[ df_log['operation'] == 'Título digitado pelo mesário' ].head(10)

,date,info,id1,info2,operation,id2
455,2022-10-02 06:00:47,INFO,67305985,VOTA,Título digitado pelo mesário,AAD567BDA8F9DECA
479,2022-10-02 06:03:53,INFO,67305985,VOTA,Título digitado pelo mesário,5E32C4175678057A
482,2022-10-02 06:04:13,INFO,67305985,VOTA,Título digitado pelo mesário,EBBD6F99E8F9402C
520,2022-10-02 06:10:05,INFO,67305985,VOTA,Título digitado pelo mesário,59652847717E91D0
559,2022-10-02 06:14:24,INFO,67305985,VOTA,Título digitado pelo mesário,8E82D2704ED8BFA9
562,2022-10-02 06:14:48,INFO,67305985,VOTA,Título digitado pelo mesário,0A1B1E0246FA20A9
583,2022-10-02 06:17:02,INFO,67305985,VOTA,Título digitado pelo mesário,4045CFA25F7431F3
603,2022-10-02 06:20:02,INFO,67305985,VOTA,Título digitado pelo mesário,3CB360C31AC3A376
635,2022-10-02 06:23:52,INFO,67305985,VOTA,Título digitado pelo mesário,F1640D5D0A282F3F
654,2022-10-02 06:29:52,INFO,67305985,VOTA,Título digitado pelo mesário,F37751601F19210D


Obtendo metadados dos logs

In [56]:
metadados_urna = df_log.query("info2 == 'GAP'")\
    [
        df_log['date'] > '2022-10-02 00:00:00'
    ]\
    [ 
        df_log['operation'].apply(
            lambda op: (
                'Modelo de Urna' in op 
                or 'Turno da UE' in op 
                or 'Título' in op
                or 'Município' in op
                or 'Zona' in op
                or 'Seção' in op
            )
        )   
    ]

metadados_urna

/tmp/ipykernel_23908/1399822644.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  metadados_urna = df_log.query("info2 == 'GAP'")\


,date,info,id1,info2,operation,id2
369,2022-10-02 04:10:35,INFO,67305985,GAP,Identificação do Modelo de Urna: UE2013,044CFC11A033F6CA
374,2022-10-02 04:10:42,INFO,67305985,GAP,Turno da UE: 1º turno,8BB7E29C04C2E4F3
375,2022-10-02 04:10:44,INFO,67305985,GAP,Identificação do Modelo de Urna: UE2013,E10D54E87435D0F6
378,2022-10-02 04:10:48,INFO,67305985,GAP,Município: 01007,A1EDECE7C05CF126
379,2022-10-02 04:10:48,INFO,67305985,GAP,Zona Eleitoral: 0009,87A01B6223D96B98
380,2022-10-02 04:10:48,INFO,67305985,GAP,Seção Eleitoral: 0001,5A96B5F2AE3D0B4C
851,2022-10-02 06:57:58,INFO,67305985,GAP,Identificação do Modelo de Urna: UE2013,BAD2D2D566965C94
856,2022-10-02 06:58:05,INFO,67305985,GAP,Turno da UE: 1º turno,C5B59FF774F0EA6A
857,2022-10-02 06:58:07,INFO,67305985,GAP,Identificação do Modelo de Urna: UE2013,8A8B60A2316DD691
860,2022-10-02 06:58:11,INFO,67305985,GAP,Município: 01007,224F64D61399160C


In [95]:
def extrair_metadado( df_metadados, like_expression, regex_pattern ):
    return df_metadados[ df_metadados['operation'].str.contains(like_expression) ]\
        .operation.str.extract( regex_pattern, expand=False ).iloc[0]

In [96]:
modelo = extrair_metadado( metadados_urna, 'Modelo de Urna', r'Modelo de Urna: (.*)' )
turno = extrair_metadado( metadados_urna, 'Turno da UE', r'Turno da UE: (.*)' )
municipio = extrair_metadado( metadados_urna, 'Município', r'Município: (.*)' )
zona = extrair_metadado( metadados_urna, 'Zona', r'Zona Eleitoral: (.*)' )
secao = extrair_metadado( metadados_urna, 'Seção', r'Seção Eleitoral: (.*)' )

print( f"""
    Modelo: {modelo}
    Turno: {turno}
    Município: {municipio}
    Zona: {zona}
    Seção: {secao}
""")


    Modelo: UE2013
    Turno: 1º turno
    Município: 01007
    Zona: 0009
    Seção: 0001



Adicionando colunas com metadados na tabela de logs

In [97]:
df_log['municipio'] = municipio
df_log['secao'] = secao
df_log['zona'] = zona
df_log['turno'] = turno
df_log['modelo'] = modelo
df_log['uf'] = 'AC' 

## Deliimitando votos

As operações que nos importam:
 - Digitação do título
 - Tentativa de reconhecimento de digital
 - Eleitor Habilitado a votar
 - Voto computado

Como isolar uma votação
1. Ordenar as operações do log de uma seção (Turno, UF, zona seção) por horário
2. Marcar cada operação de _Aguardando digitação do título_ com um contador (1 ou 2) de forma que duas seguidas não tenham o mesmo contador
3. Marcar todas as demais operações com mesmo valor da última operação de _Aguardando digitação do título_


Como isso pode ser alcançado utilizando SQL? Algumas alternativas...

1. Marcar cada operação de _Aguardando digitação do título_ com um contador (voto_id) 1 e demais com NULL
2. Utilizar `SUM(voto_id) OVER PARTITION BY (Turno, UF, zona seção) ORDER BY data` Particionar o log de uma seção (Turno, UF, zona seção) e ordenar por horário. O comportamento padrão da soma acumulada ao encontrar valores NULOS é repetir o número anterior.

Ao fim, cada voto será identificado por um número sequencial (voto_id), que pode ser utilizado para agrupar as operações de uma votação.
A identificação global do voto se dá pela combinação de Turno, UF, zona seção, voto_id.

In [125]:
operacoes = [
    'Aguardando digitação do título',
    'Título digitado pelo mesário',
    'O voto do eleitor foi computado',
    'Eleitor foi habilitado',
    'Solicita digital. Tentativa [1] de [4]',
    'Solicita digital. Tentativa [2] de [4]',
    'Solicita digital. Tentativa [3] de [4]',
    'Solicita digital. Tentativa [4] de [4]',
    'Solicitação de dado pessoal do eleitor para habilitação manual'
]

In [126]:
df_digitacao = df_log.loc[ 
    df_log['operation']=='Aguardando digitação do título',
    ['id2']
].reset_index(drop=True)

# assign 1 or 2 alternately to the rows

df_digitacao['op_id'] = (df_digitacao.index % 2 + 1).astype(int)

Filtrar operações de interesse

In [127]:
df_log_marcado = df_log.loc[ df_log['operation'].isin(operacoes) ].copy()

In [128]:
# Join the two dataframes on id2

df_log_marcado = df_log_marcado.merge(
    df_digitacao,
    how='left',
    on='id2'
)

df_log_marcado.head(5)

,date,info,id1,info2,operation,id2,municipio,secao,zona,turno,modelo,uf,op_id
0,2022-10-02 06:00:02,INFO,67305985,VOTA,Aguardando digitação do título,DBDAA51DD4DC9F2C,01007,0001,0009,1º turno,UE2013,AC,1.0
1,2022-10-02 06:00:47,INFO,67305985,VOTA,Título digitado pelo mesário,AAD567BDA8F9DECA,01007,0001,0009,1º turno,UE2013,AC,NaN
2,2022-10-02 06:00:52,INFO,67305985,VOTA,Solicita digital. Tentativa [1] de [4],103ABAFE53272BF0,01007,0001,0009,1º turno,UE2013,AC,NaN
3,2022-10-02 06:01:07,INFO,67305985,VOTA,Eleitor foi habilitado,3DD2D8E8FCFA6546,01007,0001,0009,1º turno,UE2013,AC,NaN
4,2022-10-02 06:03:12,INFO,67305985,VOTA,O voto do eleitor foi computado,EF9E02253A8457C3,01007,0001,0009,1º turno,UE2013,AC,NaN


Propagar o op_id para as outras operações

In [129]:
df_log_marcado['op_id'] = df_log_marcado['op_id'].fillna(method='ffill').astype(int)
df_log_marcado.head(25)

,date,info,id1,info2,operation,id2,municipio,secao,zona,turno,modelo,uf,op_id
0,2022-10-02 06:00:02,INFO,67305985,VOTA,Aguardando digitação do título,DBDAA51DD4DC9F2C,01007,0001,0009,1º turno,UE2013,AC,1
1,2022-10-02 06:00:47,INFO,67305985,VOTA,Título digitado pelo mesário,AAD567BDA8F9DECA,01007,0001,0009,1º turno,UE2013,AC,1
2,2022-10-02 06:00:52,INFO,67305985,VOTA,Solicita digital. Tentativa [1] de [4],103ABAFE53272BF0,01007,0001,0009,1º turno,UE2013,AC,1
3,2022-10-02 06:01:07,INFO,67305985,VOTA,Eleitor foi habilitado,3DD2D8E8FCFA6546,01007,0001,0009,1º turno,UE2013,AC,1
4,2022-10-02 06:03:12,INFO,67305985,VOTA,O voto do eleitor foi computado,EF9E02253A8457C3,01007,0001,0009,1º turno,UE2013,AC,1
5,2022-10-02 06:03:14,INFO,67305985,VOTA,Aguardando digitação do título,D1736AE5838A3DF9,01007,0001,0009,1º turno,UE2013,AC,2
6,2022-10-02 06:03:53,INFO,67305985,VOTA,Título digitado pelo mesário,5E32C4175678057A,01007,0001,0009,1º turno,UE2013,AC,2
7,2022-10-02 06:03:58,INFO,67305985,VOTA,Aguardando digitação do título,287CC15B24E59139,01007,0001,0009,1º turno,UE2013,AC,1
8,2022-10-02 06:04:13,INFO,67305985,VOTA,Título digitado pelo mesário,EBBD6F99E8F9402C,01007,0001,0009,1º turno,UE2013,AC,1
9,2022-10-02 06:04:18,INFO,67305985,VOTA,Solicita digital. Tentativa [1] de [4],90AAE36ACB93A676,01007,0001,0009,1º turno,UE2013,AC,1


In [130]:
df_log_marcado.to_csv('./data/1_AC/o00406-0100700090001/logd.csv', sep='\t', index=False)